# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ffa605ce580>
├── 'a' --> tensor([[-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539]])
└── 'x' --> <FastTreeValue 0x7ffa605ceee0>
    └── 'c' --> tensor([[ 0.3708, -0.7123, -1.4279,  0.9349],
                        [-2.0740, -0.2970,  1.4854, -0.2964],
                        [ 0.4713, -1.2394,  0.0766, -0.3328]])

In [4]:
t.a

tensor([[-1.1635,  0.1075, -1.3435],
        [-1.1074, -0.6950, -1.2539]])

In [5]:
%timeit t.a

88.4 ns ± 0.0394 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ffa605ce580>
├── 'a' --> tensor([[ 1.2471,  1.0690,  2.7027],
│                   [ 1.2866,  1.0209, -0.7131]])
└── 'x' --> <FastTreeValue 0x7ffa605ceee0>
    └── 'c' --> tensor([[ 0.3708, -0.7123, -1.4279,  0.9349],
                        [-2.0740, -0.2970,  1.4854, -0.2964],
                        [ 0.4713, -1.2394,  0.0766, -0.3328]])

In [7]:
%timeit t.a = new_value

84.9 ns ± 0.09 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539]]),
    x: Batch(
           c: tensor([[ 0.3708, -0.7123, -1.4279,  0.9349],
                      [-2.0740, -0.2970,  1.4854, -0.2964],
                      [ 0.4713, -1.2394,  0.0766, -0.3328]]),
       ),
)

In [10]:
b.a

tensor([[-1.1635,  0.1075, -1.3435],
        [-1.1074, -0.6950, -1.2539]])

In [11]:
%timeit b.a

70 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.6168,  0.3355, -0.0789],
               [-0.6358, -2.2400, -0.3661]]),
    x: Batch(
           c: tensor([[ 0.3708, -0.7123, -1.4279,  0.9349],
                      [-2.0740, -0.2970,  1.4854, -0.2964],
                      [ 0.4713, -1.2394,  0.0766, -0.3328]]),
       ),
)

In [13]:
%timeit b.a = new_value

596 ns ± 0.71 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.16 µs ± 1.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 46.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 739 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ffa60554220>
├── 'a' --> tensor([[[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]],
│           
│                   [[-1.1635,  0.1075, -1.3435],
│                    [-1.1074, -0.6950, -1.2539]]])
└── 'x' --> <FastTreeValue 0x7ff9b485ff40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 81 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ffa605ce550>
├── 'a' --> tensor([[-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539],
│                   [-1.1635,  0.1075, -1.3435],
│                   [-1.1074, -0.6950, -1.2539]])
└── 'x' --> <FastTreeValue 0x7ff9bae3c2b0>
    └── 'c' --> tensor([[ 0.3708, -0.7123, -1.4279,  0.9349],
                        [-2.0740, -0.2970,  1.4854, -0.2964],
                 

In [23]:
%timeit t_cat(trees)

36 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.6 µs ± 87.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]],
       
               [[-1.1635,  0.1075, -1.3435],
                [-1.1074, -0.6950, -1.2539]]]),
    x: Batch(
           c: tensor([[[ 0.3708, -0.7123, -1.4279,  0.9349],
                       [-2.0740, -0.2970,  1.4854, -0.2964],
                       [ 0.4713, -1.2394,  0.0766, -0.3328]],
         

In [26]:
%timeit Batch.stack(batches)

92.7 µs ± 267 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539],
               [-1.1635,  0.1075, -1.3435],
               [-1.1074, -0.6950, -1.2539]]),
    x: Batch(
           c: tensor([[ 0.3708, -0.7123, -1.4279,  0.9349],
                      [-2.0740, -0.2970,  1.4854, -0.2964],
                      [ 0.4713, -1.2394,  0.0766, -0.3328],
                      [ 0.3708, -0.7123, -1.4279,  0.9349],
                      [-2.0740, -

In [28]:
%timeit Batch.cat(batches)

171 µs ± 385 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

388 µs ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
